# Baseline CAD2 Task1

## Lyrics Intelligibility



Anecdotal mishearings of lyrics are very common, and one can find numerous examples on the internet, from websites dedicated to misheard lyrics to stand-up comedies that exploit this in a humorous way.

However, mishearing lyrics is a significant issue for those with hearing loss {cite}`greasley2020music`.

In this second round of Cadenza Challenges, we are presenting a challenge where entrants need to process a pop/rock music signal and increase its intelligibility with minimal loss of audio quality.

More details about the challenge can be found on the [Cadenza website](https://cadenzachallenge.org/docs/cadenza2/intro). 

This tutorial walks you through the process of running the lyrics intelligibility baseline using the shell interface.

## Create the environment

We first need to install the Clarity package. For this, we use the tag version of the challenge, **v6.0.0**

### Setting the Location of the Project

For convenience, we are setting an environment variable with the location of the root working directory of the project. This variable will be used in various places throughout the tutorial. Please change this value to reflect where you have installed this notebook on your system.

In [1]:
import os
os.environ["NBOOKROOT"] = os.getcwd()
os.getenv("NBOOKROOT")

'/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines'

In [2]:
from IPython.display import clear_output

import os
import sys

print("Cloning git repo...")
!git clone --depth 1 --branch v0.6.0 https://github.com/claritychallenge/clarity.git

clear_output()

Cloning git repo...
Cloning into 'clarity'...
remote: Enumerating objects: 660, done.
remote: Counting objects: 100% (660/660), done.
remote: Compressing objects: 100% (542/542), done.
remote: Total 660 (delta 74), reused 410 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (660/660), 46.44 MiB | 11.45 MiB/s, done.
Resolving deltas: 100% (74/74), done.
Note: switching to 'bebd5fce0af22ddce662e3f97871b8dd8bfac721'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [3]:
print("Installing the pyClarity...\n")
%cd clarity
%pip install -e .

sys.path.append(f'{os.getenv("NBOOKROOT")}/clarity')

clear_output()
print("Repository installed")

Repository installed


In [ ]:
%cd {os.environ['NBOOKROOT']}/clarity/recipes/cad2/task1
!pip install -r requirements.txt

## Get the demo data

The next step is to download a demo data package that will help demonstrate the process. This package has the same structure as the official data package, so it will help you understand how the files are organized.

Before continuing, it is recommended that you familiarize yourself with the data structure and content, which you can find on the [website](https://cadenzachallenge.org/docs/cadenza2/Lyric%20Intelligibility/lyric_data).

Now, let's download the data...

In [5]:
%cd {os.environ['NBOOKROOT']}/clarity
!gdown 1UqiqwYJuyC1o-C14DpVL4QYncsOGCvHF
!mv cad2_demo_data.tar.xz recipes/cad2
%cd recipes/cad2
!tar -xf cad2_demo_data.tar.xz

print("Data installed")

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity
Downloading...
From (original): https://drive.google.com/uc?id=1UqiqwYJuyC1o-C14DpVL4QYncsOGCvHF
From (redirected): https://drive.google.com/uc?id=1UqiqwYJuyC1o-C14DpVL4QYncsOGCvHF&confirm=t&uuid=422be815-311d-4732-a517-c7e88d9e3442
To: /home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/cad2_demo_data.tar.xz
100%|█████████████████████████████████████████| 132M/132M [00:01<00:00, 111MB/s]
/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad2
Data installed


## Changing working Directory

Next, we change working directory to the location of the shell scripts we wish to run.

In [6]:
%cd {os.environ['NBOOKROOT']}/clarity/recipes/cad2/task1/baseline

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad2/task1/baseline


Let's save the `root_data` path

In [7]:
root_data = f"{os.environ['NBOOKROOT']}/clarity/recipes/cad2/demo_data/cadenza_data/cad2/task1"
!ls -l {root_data}

total 8
drwxr-xr-x 3 gerardoroadabike gerardoroadabike 4096 Aug 22 15:20 audio
drwxr-xr-x 2 gerardoroadabike gerardoroadabike 4096 Aug 22 15:19 metadata


## Running the enahncement

The `enhancement` baseline employs a ConvTasNet model to separate the lyrics from the background accompaniment. This model was trained for causal and non-causal cases. The pre-trained models are stored on Huggingface. The causality is defined in the `config.yaml`.

### The config parameters

The parameters of the baseline are define in the `config.yaml` file.

---

First, it configures the paths to metadata, audio files and, location for the output files.

```yaml
path:
  root: ???   # Set to the root of the dataset
  metadata_dir: ${path.root}/metadata
  music_dir: ${path.root}/audio
  musics_file: ${path.metadata_dir}/music.valid.json
  alphas_file: ${path.metadata_dir}/alpha.json
  listeners_file: ${path.metadata_dir}/listeners.valid.json
  enhancer_params_file: ${path.metadata_dir}/compressor_params.valid.json
  scenes_file: ${path.metadata_dir}/scene.valid.json
  scene_listeners_file: ${path.metadata_dir}/scene_listeners.valid.json
  exp_folder: ./exp_${separator.causality}
```

* `path.root`: must be set to the location of the dataset.
* `exp_folder`: by default, the name of the folder it's using the causality parameter. But this can be cahnge according your requirements

---

The next parameters are the different sample rates

```yaml
input_sample_rate: 44100 # sample rate of the input mixture
remix_sample_rate: 44100 # sample rate for the output remixed signal
HAAQI_sample_rate: 24000 # sample rate for computing HAAQI score
```

The HAAQI sample rate is uses by HAAQI in the evaluation.

---
The next parameters are related to the separation and how it will operate

```yaml
separator:
  causality: causal
  device: ~
  separation:
    number_sources: 2
    segment: 6.0
    overlap: 0.1
    sample_rate: ${input_sample_rate}
```

* `separator.causality`: this is where we set the causlaity
* `separator.separation`: these parameters are used for separate large signals using fade and overlap.

--- 
The `enhancer` parameters are common parameters used by the multiband dynamic range compressor.

```yaml
enhancer:
  crossover_frequencies: [ 353.55, 707.11, 1414.21, 2828.43, 5656.85 ] # [250, 500, 1000, 2000, 4000] * sqrt(2)
  attack: [ 11, 11, 14, 13, 11, 11 ]
  release: [ 80, 80, 80, 80, 100, 100 ]
  threshold: [ -30, -30, -30, -30, -30, -30 ]
```

You are free to change the parameters if you believe it may improve the signal for the listener panel. However, the evaluation will use these parameters, and your changes may result in lower objective HAAQI scores, as this metric is based on the correlation between the enhanced and reference signals.

---

The last parameters set some of the evaluation configurations

```yaml
evaluate:
  whisper_version: base.en
  set_random_seed: True
  small_test: False
  save_intermediate: False
  equiv_0db_spl: 100
  batch_size: 1  # Number of batches
  batch: 0       # Batch number to evaluate
```

`whisper_version` is indicating what version of Whisper are we using for the intelligibility metric.

### Running enhance.py

The process goes as:
1. Loading the different metadata files into dictionaries.
2. Load the causal or non-causal separation model using the method `load_separation_model()`.
3. Create an instance of a `MultibandCompressor`.
4. Load the scenes and listeners per scenes
5. Process one scene at a time

In shell, we can call the enhancer as:
```bash
python enhance.py \
    path.root={root_data} \
    'path.listeners_file=${path.metadata_dir}/listeners.demo.json' \
    'path.enhancer_params_file=${path.metadata_dir}/compressor_params.demo.json' \
    'path.scenes_file=${path.metadata_dir}/scenes.demo.json' \
    'path.scene_listeners_file=${path.metadata_dir}/scene_listeners.demo.json' \
    'path.musics_file=${path.metadata_dir}/music.demo.json'
```

In [10]:
!python enhance.py path.root={root_data} path.listeners_file={root_data}/metadata/listeners.demo.json path.enhancer_params_file={root_data}/metadata/compressor_params.demo.json path.scenes_file={root_data}/metadata/scene.demo.json path.scene_listeners_file={root_data}/metadata/scene_listeners.demo.json path.musics_file={root_data}/metadata/music.demo.json

config.json: 100%|██████████████████████████████| 204/204 [00:00<00:00, 867kB/s]
model.safetensors: 100%|████████████████████| 43.4M/43.4M [00:00<00:00, 108MB/s]
[2024-08-23 09:14:44,110][__main__][INFO] - [0001/0002] Processing scene-listener pair: ('S50009', 'L5086')
[2024-08-23 09:15:14,287][__main__][INFO] - [0002/0002] Processing scene-listener pair: ('S50077', 'L5042')
[2024-08-23 09:15:36,094][__main__][INFO] - Enhancement completed.


---
In the prevoius cell, we processed 2 different audio signals, each for a specific target listener.
The process of a scene goes as:

1. Load the original mixture and select the requested segment
```Python
input_mixture, input_sample_rate = read_flac_signal(
    Path(config.path.music_dir)
    / songs[scene["segment_id"]]["path"]
    / "mixture.flac"
)
start_sample = int(
    songs[scene["segment_id"]]["start_time"] * config.input_sample_rate
)
end_time = int(
    (songs[scene["segment_id"]]["end_time"]) * config.input_sample_rate
)
```
---
2. Normalise to -40 dB LUFS. This is important for the compressor later.
---
3. Separate the vocals from the background
```Python
est_sources = separate_sources(
    separation_model,
    input_mixture.T,
    device=device,
    **config.separator.separation,
)
vocals, accompaniment = est_sources.squeeze(0).cpu().detach().numpy()
```
---
4. Downmix to stereo using the alpha as input parameter
```Python
enhanced_signal = downmix_signal(vocals, accompaniment, beta=alpha)
```
---
5. Aply the hearing aid amplification using the multiband compressor and the listener audiograms
```Python
enhancer.set_compressors(**mbc_params_listener["left"])
left_enhanced = enhancer(signal=enhanced_signal[0, :])

enhancer.set_compressors(**mbc_params_listener["right"])
right_enhanced = enhancer(signal=enhanced_signal[1, :])

enhanced_signal = np.stack((left_enhanced[0], right_enhanced[0]), axis=1)
```
---
6. Save the enhanced signal in FLAC. These are saved in the directory `enhanced_signals` within the experiment path `path.exp_folder` defined in the `config.yaml`.

In [17]:
!ls -l {os.environ['NBOOKROOT']}/clarity/recipes/cad2/task1/baseline/exp_causal/enhanced_signals

total 1776
-rw-rw-r-- 1 gerardoroadabike gerardoroadabike 1024842 Aug 23 09:15 S50009_L5086_A0.4_remix.flac
-rw-rw-r-- 1 gerardoroadabike gerardoroadabike  790055 Aug 23 09:15 S50077_L5042_A0.8_remix.flac


---
Let's listen to these signals.

In [12]:
from pathlib import Path
from clarity.utils.flac_encoder import read_flac_signal
from clarity.utils.signal_processing import resample
import IPython.display as ipd

audio_path = Path(os.environ['NBOOKROOT']) / "clarity/recipes/cad2/task1/baseline/exp_causal/enhanced_signals" 
audio_files = [f for f in audio_path.glob('*') if f.suffix == '.flac']

for file_to_play in audio_files:
  signal, sample_rate = read_flac_signal(file_to_play)
  signal = resample(signal, sample_rate, 16000)
  print(file_to_play.name)
  ipd.display(ipd.Audio(signal.T, rate=16000))

---
### Running evaluate.py
Now that we have enhanced audios we can use the evaluate recipe to generate HAAQI scores for the signals. The evaluation should run with the same parameters as the enhancement

In [14]:
! python evaluate.py path.root={root_data} path.listeners_file={root_data}/metadata/listeners.demo.json path.enhancer_params_file={root_data}/metadata/compressor_params.demo.json path.scenes_file={root_data}/metadata/scene.demo.json path.scene_listeners_file={root_data}/metadata/scene_listeners.demo.json path.musics_file={root_data}/metadata/music.demo.json

[2024-08-23 09:19:44,353][__main__][INFO] - Evaluating from enhanced_signals directory
[2024-08-23 09:19:45,370][__main__][INFO] - [0001/0002] Processing scene-listener pair: ('S50009', 'L5086')
[2024-08-23 09:20:09,158][root][INFO] - Severity level - SEVERE
[2024-08-23 09:20:09,158][root][INFO] - Processing {len(chans)} samples
[2024-08-23 09:20:09,165][root][INFO] - tracking fixed threshold
[2024-08-23 09:20:09,471][root][INFO] - Rescaling: leveldBSPL was 90.5 dB SPL, now 90.5 dB SPL.  Target SPL is 90.5 dB SPL.
[2024-08-23 09:20:09,471][root][INFO] - performing outer/middle ear corrections
[2024-08-23 09:20:11,986][root][INFO] - performing outer/middle ear corrections
[2024-08-23 09:20:14,542][root][INFO] - Severity level - MODERATE
[2024-08-23 09:20:14,542][root][INFO] - Processing {len(chans)} samples
[2024-08-23 09:20:14,550][root][INFO] - tracking fixed threshold
[2024-08-23 09:20:14,933][root][INFO] - Rescaling: leveldBSPL was 89.1 dB SPL, now 89.1 dB SPL.  Target SPL is 89.1 d

The evaluation scores are save in the `path.exp_folder`/scores.csv

In [23]:
import pandas as pd
pd.read_csv(f"{os.environ['NBOOKROOT']}/clarity/recipes/cad2/task1/baseline/exp_causal/scores.csv")

,scene,song,listener,haaqi_left,haaqi_right,haaqi_avg,whisper_left,whisper_rigth,whisper_be,alpha,score
0,S50009,Actions - One Minute Smile,L5086,0.93910,0.947933,0.943517,0.0,0.0,0.0,0.4,0.566110
1,S50077,Clara Berry And Wooldog - Waltz For My Victims,L5042,0.59874,0.571889,0.585314,0.5,0.5,0.5,0.8,0.517063


The HAAQI scores are compute for the left and right ear and then save the averaged.
The intelligibility score as compute for the left and right ear and then saved the better ear.

```{bibliography}
```